In [1]:
from LoopStructural.supports.structured_tetra import TetMesh
from LoopStructural.interpolators.piecewiselinear_interpolator import PiecewiseLinearInterpolator as PLI
# from LoopStructural.supports.tet_mesh import TetMesh
from LoopStructural.modelling.features.geological_feature_builder import GeologicalFeatureInterpolator
from LoopStructural.visualisation.model_visualisation import LavaVuModelViewer

import numpy as np
import lavavu
import matplotlib.pyplot as plt
import pandas as pd
import glob
%matplotlib inline

In [2]:
boundary_points = np.zeros((2,3))
boundary_points[0,0] = 548800
boundary_points[0,1] = 7816600
boundary_points[0,2] = -11010
boundary_points[1,0] = 552500
boundary_points[1,1] = 7822000
boundary_points[1,2] = -8400
steps = np.ones(3).astype(int)

steps[:] = 10

step_vector = boundary_points[1,:] - boundary_points[0,:]
# build the mesh
mesh = TetMesh(boundary_points[0,:],steps,step_vector)
# mesh.setup_mesh(boundary_points, n_tetra=50000,)

# link mesh to the interpolator 
interpolator = PLI(mesh)

In [3]:
dips = pd.read_csv('../data/Dips.csv',delimiter=';')


# import all of the csv into the same dataframe use glob to find all files matching pattern
dfs = []
for f in glob.glob('../data/*Points.csv'):
    dfs.append(pd.read_csv(f,delimiter=';'))
points = pd.concat(dfs,axis=0,ignore_index=True)

dfs = []
for f in glob.glob('../data/*Section.csv'):
    dfs.append(pd.read_csv(f,delimiter=';'))
sections = pd.concat(dfs,axis=0,ignore_index=True)

In [4]:
mesh.get_tetra_for_location(points[['X','Y','Z']].to_numpy())

(19954, 5, 4) (19954, 5, 4, 3) (19954, 5)


(array([[[ 552500., 7816600.,  -11010.],
         [ 548800., 7822000.,  -11010.],
         [ 548800., 7816600.,   -8400.],
         [ 552500., 7822000.,   -8400.]],
 
        [[ 552500., 7816600.,  -11010.],
         [ 548800., 7822000.,  -11010.],
         [ 548800., 7816600.,   -8400.],
         [ 552500., 7822000.,   -8400.]],
 
        [[ 548800., 7816600.,  -11010.],
         [ 552500., 7816600.,  -11010.],
         [ 548800., 7822000.,  -11010.],
         [ 548800., 7816600.,   -8400.]],
 
        ...,
 
        [[ 552500., 7816600.,  -11010.],
         [ 548800., 7822000.,  -11010.],
         [ 548800., 7816600.,   -8400.],
         [ 552500., 7822000.,   -8400.]],
 
        [[ 552500., 7816600.,  -11010.],
         [ 548800., 7822000.,  -11010.],
         [ 548800., 7816600.,   -8400.],
         [ 552500., 7822000.,   -8400.]],
 
        [[ 552500., 7816600.,  -11010.],
         [ 548800., 7822000.,  -11010.],
         [ 548800., 7816600.,   -8400.],
         [ 552500., 7822000

In [4]:
stratigraphy_builder = GeologicalFeatureInterpolator(
    interpolator=interpolator,
    name='stratigraphy')
solver = 'cg'
for i, r in points.iterrows():
    stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['Strati'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in sections.iterrows():
    stratigraphy_builder.add_point([r['X'],r['Y'],r['Z']],r['Strati'])#xy[0][0],xy[1][0],z],r['value'],itype=r['itype'])
for i, r in dips.iterrows():
    stratigraphy_builder.add_planar_constraint([r['X'],r['Y'],r['Z']],[r['OrientX'],r['OrientY'],r['OrientZ']])

    

In [ ]:
stratigraphy = stratigraphy_builder.build(solver='cg',cpw=1,cgw=0.1)

getting cg
eg
neigh
neighbours
iteratng
(729, 6, 3)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
26

In [ ]:
x = np.arange(0, mesh.n_cell_x)
y = np.arange(0, mesh.n_cell_y)
z = np.arange(0, mesh.n_cell_z)

c_xi, c_yi, c_zi = np.meshgrid(x,y,z)
c_xi = c_xi.flatten()
c_yi = c_yi.flatten()
c_zi = c_zi.flatten()
even_mask = (c_xi + c_yi + c_zi) % 2 == 0
# get cell corners
xi, yi, zi = mesh.cell_corner_indexes(c_xi, c_yi, c_zi)  # global_index_to_node_index(gi)
# convert to node locations
nodes = mesh.node_indexes_to_position(xi, yi, zi).T
print(even_mask.shape)
print(c_xi.shape)
print(nodes.shape)
points = np.zeros((5,4,mesh.n_cells,3))
print(points.shape)
points[:, :, even_mask, :] = nodes[:, even_mask, :][mesh.tetra_mask_even, :, :]
points[:, :, ~even_mask, :] = nodes[:, ~even_mask, :][mesh.tetra_mask, :, :]

In [ ]:
# points[:, :, even_mask, :] = nodes[:, even_mask, :][mesh.tetra_mask_even, :, :]
# points[:, :, ~even_mask, :] = nodes[:, ~even_mask, :][mesh.tetra_mask, :, :]
# changing order to points, tetra, nodes, coord
points = points.swapaxes(0, 2)
points = points.swapaxes(1, 2)
# printpoints.shape
# ps = points.reshape()
ps = points.reshape(points.shape[0]*points.shape[1],points.shape[2],points.shape[3])
print(ps.shape)

In [ ]:
m = np.array(
    [[(ps[:, 1, 0] - ps[:, 0, 0]), (ps[:, 1, 1] - ps[:, 0, 1]),
      (ps[:, 1, 2] - ps[:, 0, 2])],
     [(ps[:, 2, 0] - ps[:, 0, 0]), (ps[:, 2, 1] - ps[:, 0, 1]),
      (ps[:, 2, 2] - ps[:, 0, 2])],
     [(ps[:, 3, 0] - ps[:, 0, 0]), (ps[:, 3, 1] - ps[:, 0, 1]),
      (ps[:, 3, 2] - ps[:, 0, 2])]])
I = np.array(
    [[-1., 1., 0., 0.],
     [-1., 0., 1., 0.],
     [-1., 0., 0., 1.]])
m = np.swapaxes(m, 0, 2)
element_gradients = np.linalg.inv(m)

element_gradients = element_gradients.swapaxes(1, 2)
element_gradients = element_gradients @ I

In [ ]:
x = np.arange(0, self.n_cell_x)
y = np.arange(0, self.n_cell_y)
z = np.arange(0, self.n_cell_z)

c_xi, c_yi, c_zi = np.meshgrid(x, y, z)
c_xi = c_xi.flatten()
c_yi = c_yi.flatten()
c_zi = c_zi.flatten()
even_mask = (c_xi + c_yi + c_zi) % 2 == 0
# get cell corners
xi, yi, zi = self.cell_corner_indexes(c_xi, c_yi, c_zi)  # global_index_to_node_index(gi)
# convert to node locations
nodes = self.node_indexes_to_position(xi, yi, zi).T

# points = np.zeros((5, 4, self.n_cells, 3))
# points[:, :, even_mask, :] = nodes[:, even_mask, :][self.tetra_mask_even, :, :]
# points[:, :, ~even_mask, :] = nodes[:, ~even_mask, :][self.tetra_mask, :, :]

In [ ]:
x = np.arange(0, mesh.n_cell_x)
y = np.arange(0, mesh.n_cell_y)
z = np.arange(0, mesh.n_cell_z)

c_xi, c_yi, c_zi = np.meshgrid(x, y, z)
c_xi = c_xi.flatten()
c_yi = c_yi.flatten()
c_zi = c_zi.flatten()
even_mask = (c_xi + c_yi + c_zi) % 2 == 0
# get cell corners
xi, yi, zi = mesh.cell_corner_indexes(c_xi, c_yi, c_zi)
even_mask = (c_xi + c_yi + c_zi) % 2 == 0
print(even_mask.shape)

In [ ]:
gi = xi+yi*mesh.n_cell_x+zi*mesh.n_cell_x*mesh.n_cell_y

In [ ]:
print(gi.shape)

In [ ]:
tetras = np.zeros((c_xi.shape[0],5,4)).astype(int)
tetras[even_mask,:,:] = gi[even_mask,:][:,mesh.tetra_mask_even]
tetras[~even_mask,:,:] = gi[~even_mask,:][:,mesh.tetra_mask]

# tetras[~even_mask,:,:] = gi[~even_mask,mesh.tetra_mask]

In [ ]:
tetras.shape